In [1]:
import pandas as pd
import datetime
import os
import snscrape.modules.twitter as sntwitter
import numpy as np
import time
import random
import sys
import queue
import threading
from IPython.display import clear_output as clear

In [2]:
date_series = [f"D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-0{month}\\tweet_ids_day_2022-{month}-{date}.txt"
              for month in range(2,10) for date in range(1,32)]
dates = []
for date_data in date_series:
    if os.path.exists(date_data):
        with open(date_data,'r') as f:
            dates.append(date_data)

In [3]:
dates

['D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-02\\tweet_ids_day_2022-2-22.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-02\\tweet_ids_day_2022-2-23.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-02\\tweet_ids_day_2022-2-24.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-02\\tweet_ids_day_2022-2-25.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-02\\tweet_ids_day_2022-2-26.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-02\\tweet_ids_day_2022-2-27.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-02\\tweet_ids_day_2022-2-28.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-03\\tweet_ids_day_2022-3-1.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-03\\tweet_ids_day_2022-3-2.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-03\\tweet_ids_day_2022-3-3.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-03\\tweet_ids_day_2022-3-4.txt',
 'D:\\hrfan\\RussoUkrainianWar_Dataset-main\\2022-03\\tweet_ids_day_2022-3-5.txt

In [4]:
sem=threading.Semaphore(50) #限制线程的最大数量50，可以根据电脑性能调整线程数
def tweetscraper(j, tweets_list):
    with sem:
        clear(wait=True)#先清除输出区域，不然会一直打印造成ipynb文件过大
        try:
            for i,tweet in enumerate(sntwitter.TwitterTweetScraper(j).get_items()):
                tweets_list.append([
                    #这里有些用的是老版本的变量名，会提示使用新的名称，但由于我最开始就用老名称爬就没改
                    tweet.date, tweet.url, tweet.id, tweet.content, tweet.replyCount, tweet.retweetCount, tweet.likeCount,
                    tweet.quoteCount, tweet.conversationId, tweet.lang, tweet.source, tweet.sourceUrl, tweet.outlinks, tweet.media,
                    tweet.retweetedTweet, tweet.quotedTweet, tweet.inReplyToTweetId, tweet.inReplyToUser, tweet.mentionedUsers, 
                    tweet.coordinates, tweet.place, tweet.hashtags, tweet.cashtags, #tweet相关的字段
                    tweet.user.username, tweet.user.displayname, tweet.user.id, tweet.user.rawDescription, tweet.user.descriptionUrls,
                    tweet.user.verified, tweet.user.created,tweet.user.followersCount, tweet.user.friendsCount, tweet.user.statusesCount, 
                    tweet.user.favouritesCount,tweet.user.listedCount, tweet.user.mediaCount, tweet.user.location, tweet.user.protected,
                    tweet.user.linkUrl,tweet.user.profileImageUrl, tweet.user.profileBannerUrl, tweet.user.label ##user相关的字段
                    ])
            time.sleep(1)
            print(len(tweets_list), "tweets have been scraped", flush = True)
            time.sleep(0.1)
        except:
            print("a mistake have been passed", flush = True)
            time.sleep(0.1)

In [7]:
#开始爬！

for i in range(len(dates)):
    name = dates[i] +'_tweets.csv'#输出的文件地址
    if os.path.exsits(name):#如果已经爬过了就跳过
        print(name, ' has been scraped, passed.', flush=True)
        continue
        
    else:
        testset = []
        with open(dates[i], 'r') as f:
            for line in f:
                testset.append(line.strip('\\n').split(','))
        test = []
        for j in testset:
            test.append(int(j[0]))
        # create multi-threads to scrape
        threads = []
        tweets_list = []
        print(dates[i], 'scraper begins')

        for j in test:
            threads.append(
                threading.Thread(target = tweetscraper, args=(j, tweets_list))
                )

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

        print(dates[i], 'has been scraped')

        tweets_df = pd.DataFrame(tweets_list, columns=[
                                                    'Datetime', 'Tweet_url', 'Tweet_id', 'Tweet_content', 'Tweet_reply_count', 'Tweet_retweet_count',
                                                    'Tweet_like_count', 'Tweet_quote_count', 'Tweet_conversation_id', 'Tweet_language', 'Tweet_source', 
                                                    'Tweet_source_url', 'Tweet_links', 'Tweet_mdeia', 'Tweet_retweeted_tweet', 'Tweet_quoted_tweet', 
                                                    'Tweet_inReplyToTweetId','Tweet_inReplyToUser','Tweet_mentioned_users','Tweet_coordinates', 'Tweet_place', 
                                                    'Tweet_hashtags', 'Tweet_cashtags',#tweet 相关的字段
                                                    'Username','User_displayname', 'User_id', 'User_profile_description', 'User_description_link', 'User_verified',
                                                    'User_created', 'User_followers_count', 'User_friends_count', 'User_statuses_count', 'User_favourites_count',
                                                    'User_listed_count', 'User_media_count', 'User_location', 'User_protected', 'User_profile_link', 'User_profile_image_url',
                                                    'User_profile_banner_url', 'User_label' #user 相关的字段
                                                  ])

        tweets_df.to_csv(name)